In [8]:
from ollama import chat
from ollama import ChatResponse
import torch
import json
import os

In [9]:
def load_json(filepath):
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {filepath}")
        return None

In [1]:
def save_json(data, filepath):
    try:
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=4)
        print(f"Data saved successfully to {filepath}")
    except Exception as e:
        print(f"Error saving data to {filepath}: {e}")

In [11]:
system_prompt_dict = {}

In [12]:
system_prompt_dict["PHP"] = """
You are an expert PHP code analyst. Your task is to dissect a given PHP code snippet and provide a structured JSON response detailing all of its components and overall functionality. You must strictly adhere to the following JSON format:
{
  "output": {
    "programming_language": "PHP",
    "components": [
      {
        "component_type": "<TYPE_OF_PHP_COMPONENT>",
        "component_name": "<NAME_OF_COMPONENT_IF_APPLICABLE>",
        "component_code": "<THE_ACTUAL_PHP_CODE_OF_THE_COMPONENT>",
        "component_description": "<DETAILED_DESCRIPTION_OF_COMPONENT_FUNCTIONALITY>"
      },
      { /* ... more components ... */ }
    ],
    "overall_description": "<DETAILED_SUMMARY/DESCRIPTION_OF_THE_ENTIRE_PHP_CODE_FUNCTIONALITY>"
  }
}

**Instructions:**
1. Identify the Programming Language: The script is written in PHP.
2. Component Types: Use appropriate component types, for example, VARIABLE_DECLARATION, FUNCTION_DEFINITION, CLASS_DEFINITION, METHOD_DEFINITION, CONDITIONAL_STATEMENT, LOOP, DATABASE_QUERY, FORM_HANDLING, ERROR_HANDLING, NAMESPACE_DECLARATION, USE_STATEMENT.
3. Component Names: Provide the correct identifier for each component (function name, class name, variable name, etc.), or NULL if not applicable.
4. Component Code: Include the complete, unmodified PHP code for each component.
5. Component Descriptions: Provide a detailed, technical explanation of what each component does, including server-side scripting, form validation, session management, and database interaction.
6. Overall Description: Provide a detailed summary of the entire PHP code, explaining its purpose, architecture, and how components interact.
7. Strict JSON Output: Your ENTIRE response must be ONLY the valid JSON object. Do not include any explanations, introductions, or additional text outside the JSON structure.

Analyze the following PHP code properly and return ONLY the JSON response with no additional text:
"""

In [13]:
def string_to_json(input_string):
    try:
        data = json.loads(input_string)
        return data, None
    except json.JSONDecodeError as e:
        return None, str(e)

In [14]:
def make_data(data, system_prompt):
    output_data = {}

    for key, value in data.items():
        model_name = "qwen2.5-coder:32b"

        response: ChatResponse = chat(model=model_name, messages=[
            {
                'role': 'system',
                'content': system_prompt,
            },
            { 
                'role': 'user',
                'content': value,
            }
        ])

        output_data[key] = {
            "input" : value,
            "output" : response['message']['content']
        }

        print("Processed:", key)
            
    return output_data

In [15]:
data = load_json("php_train.json")

num_keys = 25
if not isinstance(data, dict):
    raise TypeError("Input must be a dictionary.")

if not isinstance(num_keys, int) or num_keys < 0:
    raise ValueError("num_keys must be a non-negative integer.")

sliced_dict = {}
count = 0
for key, value in data.items():
    if count < num_keys:
        sliced_dict[key] = value
        count += 1
    else:
        break

del data
del count

In [16]:
print("Length of sliced dictionary:", len(sliced_dict))

Length of sliced dictionary: 25


In [17]:
output = make_data(sliced_dict, system_prompt_dict["PHP"])

Processed: php_0
Processed: php_1
Processed: php_2
Processed: php_3
Processed: php_4
Processed: php_5
Processed: php_6
Processed: php_7
Processed: php_8
Processed: php_9
Processed: php_10
Processed: php_11
Processed: php_12
Processed: php_13
Processed: php_14
Processed: php_15
Processed: php_16
Processed: php_17
Processed: php_18
Processed: php_19
Processed: php_20
Processed: php_21
Processed: php_22
Processed: php_23
Processed: php_24


In [21]:
for k, v in output.items():
    print(f"Input: \n{v['input']}\n")
    print(f"Output: \n{v['output']}\n")
    print("-" * 80)

Input: 
<filename>extensions/vscode-colorize-tests/test/colorize-fixtures/issue-76997.php
<hello></hello>


Output: 
{
  "output": {
    "programming_language": "PHP",
    "components": [
      {
        "component_type": "HTML_TAG",
        "component_name": "hello",
        "component_code": "<hello></hello>",
        "component_description": "This is an HTML tag. In the context of PHP, this line does not execute any PHP code but rather outputs an opening and closing 'hello' tag as plain HTML. It serves no functionality within PHP itself."
      }
    ],
    "overall_description": "The provided file contains what appears to be a snippet that includes an HTML tag '<hello></hello>'. There is no actual PHP code present in this snippet; it does not perform any server-side processing, database interaction, or any other function typical of PHP scripts. The only content is an HTML-like structure which would render as such if included in a web page."
  }
}

----------------------------------

In [23]:
import pickle
# Save the output to a pickle file
with open("data.pickle", "wb") as file:
    pickle.dump(output, file)

In [2]:
import pickle
# Load the output from the pickle file
with open("data.pickle", "rb") as file:
    loaded_output = pickle.load(file)
print("Loaded output from pickle file:")
for i, (k, v) in enumerate(loaded_output.items()):
    print(f"Input {i}: \n{v['input']}\n")
    print(f"Output {i}: \n{v['output']}\n")
    print("-" * 80)

Loaded output from pickle file:
Input 0: 
<filename>extensions/vscode-colorize-tests/test/colorize-fixtures/issue-76997.php
<hello></hello>


Output 0: 
{
  "output": {
    "programming_language": "PHP",
    "components": [
      {
        "component_type": "HTML_TAG",
        "component_name": "hello",
        "component_code": "<hello></hello>",
        "component_description": "This is an HTML tag. In the context of PHP, this line does not execute any PHP code but rather outputs an opening and closing 'hello' tag as plain HTML. It serves no functionality within PHP itself."
      }
    ],
    "overall_description": "The provided file contains what appears to be a snippet that includes an HTML tag '<hello></hello>'. There is no actual PHP code present in this snippet; it does not perform any server-side processing, database interaction, or any other function typical of PHP scripts. The only content is an HTML-like structure which would render as such if included in a web page."
  }
}

In [ ]:
loaded_output["php_2"]["output"] = loaded_output["php_2"]["output"][:loaded_output["php_2"]["output"].index("');")]

ValueError: substring not found

In [5]:
loaded_output["php_9"]["output"] = loaded_output["php_9"]["output"][:loaded_output["php_9"]["output"].index("```")-1]

In [6]:
loaded_output["php_21"]["output"] = loaded_output["php_21"]["output"] + "\n}"

In [7]:
# After manual verification

final_data = {}
remove_ind_num = [6, 8, 12]
for i, (k, v) in enumerate(loaded_output.items()):
    if i not in remove_ind_num:
        print(f"Keeping index {i} in output.")
        final_data[k] = v

Keeping index 0 in output.
Keeping index 1 in output.
Keeping index 2 in output.
Keeping index 3 in output.
Keeping index 4 in output.
Keeping index 5 in output.
Keeping index 7 in output.
Keeping index 9 in output.
Keeping index 10 in output.
Keeping index 11 in output.
Keeping index 13 in output.
Keeping index 14 in output.
Keeping index 15 in output.
Keeping index 16 in output.
Keeping index 17 in output.
Keeping index 18 in output.
Keeping index 19 in output.
Keeping index 20 in output.
Keeping index 21 in output.
Keeping index 22 in output.
Keeping index 23 in output.
Keeping index 24 in output.


In [8]:
final_data.keys()

dict_keys(['php_0', 'php_1', 'php_2', 'php_3', 'php_4', 'php_5', 'php_7', 'php_9', 'php_10', 'php_11', 'php_13', 'php_14', 'php_15', 'php_16', 'php_17', 'php_18', 'php_19', 'php_20', 'php_21', 'php_22', 'php_23', 'php_24'])

In [9]:
# Save the output to a JSON file
import json
output_json_path = "final_kotlin_data.json"
try:
    save_json(final_data, output_json_path)
except Exception as e:
    print(f"Error saving final data to JSON: {e}")

Data saved successfully to final_kotlin_data.json
